# Loop data, Part 5: Plot day to day comparisons

What I'm trying to do this time:

- plot several days of data on top of each other
- have the user (well... me) select the number of days to show (I guess start and end date) and also select the time window of interest
- eventually (probably not in this notebook but we'll see) have some kind of hover tool (or something else??) to look at treatment info for the selected time period for a selected day. 

In [4]:
import sys
sys.path.append("../")

from mdb_tools import load_data as ld
from mdb_tools import schemas
from mdb_tools import loop_stats as oop
from mdb_tools import sugar_plots as sp

import pandas as pd
import numpy as np

from datetime import datetime
import pytz

import matplotlib.pyplot as plt
%matplotlib widget

In [2]:
yml_secrets_file = '../../secrets/mdb_secrets.yml'

# Access the database using the yml secrets file, and get a specific set of "collections"
col_entries, col_treatments, col_profile, col_devicestatus = ld.get_collections(yml_secrets_file)

# Grab schemas
entries_schema, treatments_schema, devicestatus_schema = schemas.mdb_schemas()

# Load
df_entries0 = col_entries.find_pandas_all({}, schema=entries_schema)
df_treatments = col_treatments.find_pandas_all({}, schema=treatments_schema)
df_devicestatus = col_devicestatus.find_pandas_all({}, schema=devicestatus_schema)

# Remove duplicate entries from cgm date - keep only loop for now.
df_entries = df_entries0[df_entries0["device"]=="loop://Dexcom/G6/21.0"].copy()

In [3]:
# Convert the time string to datetime format (not converting time zone this time!)
df_entries["time"] = pd.to_datetime(df_entries["dateString"]).dt.tz_convert('US/Eastern')
df_entries.index = df_entries["time"]
df_entries.sort_index(inplace=True)

In [5]:
# Set up my start/end dates and start/end times
date_start = '2023-10-25'
date_end = '2023-11-01'

# start and end time in hours/time of day
time_start = 6
time_end = 12

In [6]:
# Extract only the dates of interest
df_sub = df_entries.loc[date_start:date_end]
